# Imports

In [ ]:
!pip install timm

In [ ]:
!pip install pytorch-metric-learning

In [ ]:
import numpy as np 
import pandas as pd
import random
import os
import math
import time

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn.utils import class_weight
from PIL import Image as pil_image
from tqdm import tqdm
import scipy

import matplotlib
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader

import timm
from timm.optim import Lookahead, RAdam
from pytorch_metric_learning import miners, losses, samplers , distances, regularizers 

# Global

In [ ]:
IMG_SIZE = 256
SEED = 42
N_MATCHES = 5

PROJECT_FOLDER = "../input/hotel-id-to-combat-human-trafficking-2022-fgvc9/"
DATA_FOLDER = "../input/nopadding256/"
IMAGE_FOLDER = DATA_FOLDER + "images/"
OUTPUT_FOLDER = ""

train_df = pd.read_csv(os.path.join(DATA_FOLDER, 'train_no_padding_256.csv'))

In [ ]:
print(os.listdir(PROJECT_FOLDER))
print(len(os.listdir(IMAGE_FOLDER)))

# Helper functions - seed and metric calculator

In [ ]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

# Dataset and transformations

In [ ]:
import albumentations as A
import albumentations.pytorch as APT
import cv2 

# used for training dataset - augmentations and occlusions
train_transform = A.Compose([
    A.Transpose(p=0.5),
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.RandomBrightnessContrast(p=0.75),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, border_mode=0, p=0.85),
    
    A.CoarseDropout(p=1., max_holes=1, 
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2, 
                    fill_value=(255,0,0)),# simulating occlusions in test data

    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

# used for validation dataset - only occlusions
val_transform = A.Compose([
    A.CoarseDropout(p=1., max_holes=1, 
                    min_height=IMG_SIZE//4, max_height=IMG_SIZE//2,
                    min_width=IMG_SIZE//4,  max_width=IMG_SIZE//2, 
                    fill_value=(255,0,0)),# simulating occlusions
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

# no augmentations
base_transform = A.Compose([
    A.ToFloat(),
    APT.transforms.ToTensorV2(),
])

In [ ]:
class HotelTrainDataset:
    def __init__(self, data, transform=None, data_path="train_images/"):
        self.data = data
        self.data_path = data_path
        self.transform = transform

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        record = self.data.iloc[idx]
        image_path = self.data_path + record["image_id"]
        image = np.array(pil_image.open(image_path)).astype(np.uint8)

        if self.transform:
            transformed = self.transform(image=image)
            image = transformed["image"]
        
        return {
            "image" : image,
            "target" : record['hotel_id_code'],
        }

# Model

In [ ]:
class EmbeddingNet(nn.Module):
    def __init__(self, n_classes=100, embed_size=64, backbone_name="efficientnet_b0"):
        super(EmbeddingNet, self).__init__()
        
        self.embed_size = embed_size
        self.backbone = timm.create_model(backbone_name, pretrained=True)
        in_features = self.backbone.get_classifier().in_features

        fc_name, _ = list(self.backbone.named_modules())[-1]
        if fc_name == 'classifier':
            self.backbone.classifier = nn.Identity()
        elif fc_name == 'head.flatten':
            self.backbone.head.fc = nn.Identity()
        elif fc_name == 'fc':
            self.backbone.fc = nn.Identity()
        elif fc_name == 'head':
            self.backbone.head = nn.Identity()
        else:
            raise Exception("unknown classifier layer: " + fc_name)
        
        self.post = nn.Sequential(
            nn.utils.weight_norm(nn.Linear(in_features, self.embed_size*2), dim=None),
            nn.BatchNorm1d(self.embed_size*2),
            nn.Dropout(0.2),
            nn.utils.weight_norm(nn.Linear(self.embed_size*2, self.embed_size)),
        )

        self.classifier = nn.Sequential(
            nn.BatchNorm1d(self.embed_size),
            nn.Dropout(0.2),
            nn.Linear(self.embed_size, n_classes),
        )
        
    def embed_and_classify(self, x):
        x = self.forward(x)
        return x, self.classifier(x)

    def forward(self, x):
        x = self.backbone(x)
        x = x.view(x.size(0), -1)
        x = self.post(x)
        return x

# Model helper functions

In [ ]:
def get_embeds(loader, model, bar_desc="Generating embeds"):
    targets_all = []
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            target = sample['target'].to(args.device)
            output = model(input)

            targets_all.extend(target.cpu().numpy())
            outputs_all.extend(output.detach().cpu().numpy())
            
    return targets_all, outputs_all

def get_prediction(loader, model, bar_desc="Generating predictions"):
    targets_all = []
    outputs_all = []
    
    model.eval()
    with torch.no_grad():
        t = tqdm(loader, desc=bar_desc)
        for i, sample in enumerate(t):
            input = sample['image'].to(args.device)
            target = sample['target'].to(args.device)
            _, output = model.embed_and_classify(input)
            output = torch.argsort(torch.sigmoid(output), descending=True)

            targets_all.extend(target.cpu().numpy())
            outputs_all.extend(output.detach().cpu().numpy())
            
    return targets_all, outputs_all

In [ ]:
def get_distance_matrix(embeds, base_embeds, distance_func):
    distance_matrix = []
    base_embeds = torch.Tensor(base_embeds)
    embeds_dataset = torch.utils.data.TensorDataset(torch.Tensor(embeds))
    embeds_dataloader = DataLoader(embeds_dataset, num_workers=2, batch_size=1024, shuffle=False)
    
    t = tqdm(embeds_dataloader)
    for i, sample in enumerate(t): 
        distances = distance_func(sample[0], base_embeds)
        distance_matrix.extend(distances.numpy())
        
    return np.array(distance_matrix)

In [ ]:
def save_checkpoint(model, scheduler, optimizer, epoch, name, loss=None, score=None):
    checkpoint = {"epoch": epoch,
                  "model": model.state_dict(),
                  "scheduler": scheduler.state_dict(),
                  "optimizer": optimizer.state_dict(),
                  "loss": loss,
                  "score": score,
                  }

    torch.save(checkpoint, f"{OUTPUT_FOLDER}checkpoint-{name}.pt")


def load_checkpoint(model, scheduler, optimizer, name):
    checkpoint = torch.load(f"{OUTPUT_FOLDER}checkpoint-{name}.pt")

    model.load_state_dict(checkpoint["model"])
    scheduler.load_state_dict(checkpoint["scheduler"])
    # optimizer.load_state_dict(checkpoint["optimizer"])

    return model, scheduler, optimizer, checkpoint["epoch"]

In [ ]:
def iterate_loader(loader, epochs):
    loader.dataset.fake_load = True
    with torch.no_grad():
        for i in range(epochs):
            t = tqdm(loader, desc=f"Iterating loader {i+1}/{epochs}")
            for j, sample in enumerate(t):
                images = sample['image']
                targets = sample['target']

    loader.dataset.fake_load = False

In [ ]:
def train_epoch(args, model, loader, criterion, optimizer, scheduler, epoch):
    losses = []
    targets_all = []
    outputs_all = []
    
    model.train()
    t = tqdm(loader)
    
    for i, sample in enumerate(t):
        optimizer.zero_grad()
        
        images = sample['image'].to(args.device)
        targets = sample['target'].to(args.device)
        
        embeds, outputs = model.embed_and_classify(images)
        loss = criterion(outputs, targets)
        
        loss.backward()
        optimizer.step()
        
        if scheduler:
            scheduler.step()
                
        losses.append(loss.item())
        targets_all.extend(targets.cpu().numpy())
        outputs_all.extend(torch.sigmoid(outputs).detach().cpu().numpy())

        score = np.mean(targets_all == np.argmax(outputs_all, axis=1))
        desc = f"Epoch {epoch}/{args.epochs} - Train loss:{loss:0.4f}, score: {score:0.4f}"
        t.set_description(desc)
        
    return np.mean(losses), score


def test_closest_match(base_df, base_embeds, valid_targets, valid_embeds, model, distance_func, closest, n_matches=5):
    distance_matrix = get_distance_matrix(valid_embeds, base_embeds, distance_func)

    preds = []
    N_val = len(valid_embeds)
    for i in tqdm(range(N_val), total=N_val, desc="Getting closest match"):
        tmp_df = base_df.copy()
        tmp_df["distance"] = distance_matrix[i]
        tmp_df = tmp_df.sort_values(by=["distance", "hotel_id"], ascending=closest).reset_index(drop=True)
        preds.extend([tmp_df["hotel_id_code"].unique()[:n_matches]])

    y = np.repeat([valid_targets], repeats=n_matches, axis=0).T
    preds = np.array(preds)
    acc_top_1 = (preds[:, 0] == valid_targets).mean()
    acc_top_5 = (preds == y).any(axis=1).mean()
    print(f"Accuracy: {acc_top_1:0.4f}, top 5 accuracy: {acc_top_5:0.4f}")
    return preds, distance_matrix


def test(base_loader, valid_loader, model, distance_func, closest):
    base_targets, base_embeds = get_embeds(base_loader, model, "Generating embeds for train")
    valid_targets, valid_embeds = get_embeds(valid_loader, model, "Generating embeds for test")
    val_preds, distance_matrix = test_closest_match(base_loader.dataset.data, base_embeds, valid_targets, valid_embeds, model, distance_func, closest)

    return base_embeds, valid_embeds, base_targets, valid_targets, val_preds, distance_matrix

# Prepare data

In [ ]:
def sample_data(n_hotels, min_images, max_images):
    data_df = pd.read_csv(PROJECT_FOLDER + "train.csv", parse_dates=["timestamp"])
    sample_df = data_df.groupby("hotel_id").filter(lambda x: (x["image"].nunique() > min_images) & (x["image"].nunique() < max_images))
    sample_df["hotel_id_code"] = sample_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)
    sample_df = sample_df[sample_df["hotel_id_code"] < n_hotels]

    print(f"Subsample with {len(sample_df.hotel_id.unique())} hotels out of {len(data_df.hotel_id.unique())}" + 
          f" with total {len(sample_df)} images ({len(sample_df) / len(data_df) * 100:0.2f} %)")
    
    return sample_df

In [ ]:
data_df = pd.read_csv(DATA_FOLDER + "train_no_padding_256.csv")
# encode hotel ids
data_df["hotel_id_code"] = data_df["hotel_id"].astype('category').cat.codes.values.astype(np.int64)

In [ ]:
fig = go.Figure()
fig.add_trace(go.Histogram(x=data_df["hotel_id_code"]))
fig.update_xaxes(type="category")
fig.show()

# Train and evaluate

In [ ]:
data_df

In [ ]:
hotel_image_count = data_df.groupby("hotel_id")["image_id"].count()
    # hotels that have more images than samples for validation
valid_hotels = hotel_image_count[hotel_image_count > 1]
    # data that can be split into train and val set
valid_data = data_df[data_df["hotel_id"].isin(valid_hotels.index)]
    # if hotel had less than required val_samples it will be only in the train set
valid_df = valid_data.groupby("hotel_id").sample(1, random_state=SEED)
train_df = data_df[~data_df["image_id"].isin(valid_df["image_id"])]

In [ ]:
train_df

In [ ]:
def train_and_validate(args, data_df):
    model_name = f"classification-model-{args.backbone_name}-{IMG_SIZE}x{IMG_SIZE}-{args.embed_size}embeds-{args.n_classes}hotels"
    print(model_name)
    # SEED and split
    seed_everything(seed=SEED)
    hotel_image_count = data_df.groupby("hotel_id")["image_id"].count()
    # hotels that have more images than samples for validation
    valid_hotels = hotel_image_count[hotel_image_count > 1]
    # data that can be split into train and val set
    valid_data = data_df[data_df["hotel_id"].isin(valid_hotels.index)]
    # if hotel had less than required val_samples it will be only in the train set
    valid_df = valid_data.groupby("hotel_id").sample(1, random_state=SEED)
    train_df = data_df[~data_df["image_id"].isin(valid_df["image_id"])]

    # create model
    model = EmbeddingNet(args.n_classes, args.embed_size, args.backbone_name)
    model = model.to(args.device)

    train_dataset = HotelTrainDataset(train_df, train_transform, data_path=IMAGE_FOLDER)
    train_loader  = DataLoader(train_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=True, drop_last=True)
    valid_dataset = HotelTrainDataset(valid_df, val_transform, data_path=IMAGE_FOLDER)
    valid_loader  = DataLoader(valid_dataset, num_workers=args.num_workers, batch_size=args.batch_size, shuffle=False)
    # base dataset for image similarity search
    base_dataset  = HotelTrainDataset(train_df, base_transform, data_path=IMAGE_FOLDER)
    base_loader   = DataLoader(base_dataset, num_workers=args.num_workers, batch_size=args.batch_size*4, shuffle=False)
    print(f"Base: {len(base_dataset)}\nValidation: {len(valid_dataset)}")

    distance = distances.CosineSimilarity()
    criterion = nn.CrossEntropyLoss()
    optimizer = Lookahead(torch.optim.AdamW(model.parameters(), lr=args.lr), k=3)

    scheduler = torch.optim.lr_scheduler.OneCycleLR(
                    optimizer,
                    max_lr=args.lr,
                    epochs=args.epochs,
                    steps_per_epoch=len(train_loader),
                    div_factor=10,
                    final_div_factor=1,
                    pct_start=0.1,
                    anneal_strategy="cos",
                )
    
    start_epoch = 1

    if args.continue_from_checkpoint:
        model, scheduler, optimizer, last_epoch = load_checkpoint(model, scheduler, optimizer, model_name)
        iterate_loader(train_loader, last_epoch)
        start_epoch = start_epoch + last_epoch

    torch.cuda.empty_cache()

    for epoch in range(start_epoch, args.epochs+1):
        train_loss, train_score = train_epoch(args, model, train_loader, criterion, optimizer, scheduler, epoch)
        save_checkpoint(model, scheduler, optimizer, epoch, model_name, train_loss, train_score)
        if (epoch == 1):
            _ = test(base_loader, valid_loader, model, distance, closest=False)

    base_embeds, valid_embeds, base_targets, valid_targets, val_preds, distance_matrix = test(base_loader, valid_loader, model, distance, closest=False)

In [ ]:
class args:
    epochs = 10
    lr = 1e-3
    batch_size = 32
    num_workers = 2
    embed_size = 2048
    val_samples = 1
    backbone_name="densenet161"
    n_classes = data_df["hotel_id_code"].nunique()
    device = ('cuda' if torch.cuda.is_available() else 'cpu')
    continue_from_checkpoint = False

train_and_validate(args, data_df)